In [1]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [18]:
data <- read_csv2("Absenteeism_at_work.csv")
colnames(data) <- make.names(colnames(data))

data_tidy <- data |>
    mutate_at(c("ID", "Reason.for.absence", "Month.of.absence", "Day.of.the.week", "Seasons", 
                "Disciplinary.failure", "Education", "Social.drinker", "Social.smoker"), as.factor) |>
    select(-Reason.for.absence,-Month.of.absence,-Day.of.the.week,-Seasons,-Hit.target,-Weight,-Height,-Work.load.Average.day,-Disciplinary.failure) |>
    arrange(ID)

data_total_hours <- data_tidy |>
    group_by(ID) |>
    summarize(sum(Absenteeism.time.in.hours)) |>
    select(-ID)

data_1_ID_per_row <- data_tidy |>
    mutate(across(everything(), as.numeric)) |>
    group_by(ID) |>
    summarize(across(Transportation.expense:Absenteeism.time.in.hours, mean))
 
data_by_days <- data |>
    group_by(ID) |>
    summarize(n = n()) |>
    select(-ID)

all_data <- cbind(data_1_ID_per_row, data_total_hours, data_by_days) |>
        select(-Absenteeism.time.in.hours) |>
        rename(Days.of.absence = n, Hours.of.absence = "sum(Absenteeism.time.in.hours)") |>
        mutate(ID=as_factor(ID),Social.drinker=as_factor(Social.drinker),
               Social.smoker=as_factor(Social.smoker),Education=as_factor(Education))

#all_data
data_1 <- data |> #select(data, Reason.for.absence,Age) |>
        group_by(Age) |>
        Summarize(n=n())

data_1

#plot <- ggplot(data,aes(y= Reason.for.absence,x=Age)) +
 #   geom_point() #+
    #facet_grid(rows=vars(Reason.for.absence))
#plot

data_selected <- select(data,Reason.for.absence,Absenteeism.time.in.hours,Body.mass.index) |>
        mutate(across(Reason.for.absence, as.factor)) 
 #   group_by(Reason.for.absence)|>
 #   summarize(Absenteeism.time.in.hours = mean(Absenteeism.time.in.hours), 
   #           Body.mass.index = mean(Body.mass.index)) 

data_table <- data_selected |>
    group_by(Reason.for.absence)|>
    summarize(n=n()) |>
    arrange(desc(n))
data_table

data <- filter(data, Reason.for.absence == "23","28","27","13","0","19","22","26","25","11")

#data_table <- select(data,Reason.for.absence,Absenteeism.time.in.hours,Body.mass.index) |>
#    mutate(across(Reason.for.absence, as.factor))  |>
#    group_by(Absenteeism.time.in.hours)|>
#    summarize(Absenteeism.time.in.hours = mean(Absenteeism.time.in.hours), Body.mass.index = mean(Body.mass.index), Age = mean(Age))|>
#    arrange(Reason.for.absence)
#data_table

data_mean_time <- select(data,Reason.for.absence,Absenteeism.time.in.hours,Body.mass.index) |>
        mutate(across(Reason.for.absence, as.factor))  |>
    group_by(Reason.for.absence)|>
    summarize(Absenteeism.time.in.hours = mean(Absenteeism.time.in.hours))|>
    arrange(Absenteeism.time.in.hours) |>
    tibble::rowid_to_column("ID")
data_mean_time

data_mean_BMI <- select(data,Reason.for.absence,Absenteeism.time.in.hours,Body.mass.index) |>
        mutate(across(Reason.for.absence, as.factor))  |>
    group_by(Reason.for.absence)|>
    summarize(Body.mass.index = mean(Body.mass.index)) |>
    arrange(Body.mass.index) |>
    tibble::rowid_to_column("ID")
data_mean_BMI

data_mean_Age <- select(data,Reason.for.absence,Absenteeism.time.in.hours,Body.mass.index,Age) |>
        mutate(across(Reason.for.absence, as.factor))  |>
    group_by(Reason.for.absence)|>
    summarize(Age = mean(Age)) |>
    arrange(Age) |>
    tibble::rowid_to_column("ID")
data_mean_Age

data_plot_time <- ggplot(data_mean_time, aes(x=ID,y=Absenteeism.time.in.hours)) +
    geom_point()

data_plot_BMI <- ggplot(data_mean_BMI, aes(x=ID,y=Body.mass.index)) +
    geom_point()

data_plot_Age <- ggplot(data_mean_Age, aes(x=ID,y=Age)) +
    geom_point()

data_plot_time
data_plot_BMI
data_plot_Age 

data_split <- initial_split(data_selected, prop = 0.75, strata = Reason.for.absence)  
data_train <- training(data_split)   
data_test <- testing(data_split)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 740 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (20): ID, Reason for absence, Month of absence, Day of the week, Seasons...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in Summarize(group_by(data, Age), n = n()): could not find function "Summarize"
